## PART 1 (step 1 - 8)

In [27]:
!ls -l
#!cat mystuff.py

total 593
-rw-r--r-- 1 user user   22707 May 19 02:00 Chong_Project_01.ipynb
-rw-r--r-- 1 user user    5607 May 17 14:41 Readme.txt
drwxr-xr-x 2 user user       3 May 17 14:49 __pycache__
-rw-r--r-- 1 user user   45029 May 17 14:41 data_banknote_authentication.txt
-rw-r--r-- 1 user user   57569 May 17 14:41 day.csv
-rw-r--r-- 1 user user    1838 May 17 14:41 description.md
-rw-r--r-- 1 user user 1156736 May 17 14:41 hour.csv
-rw-r--r-- 1 user user    1858 May 17 14:41 mystuff.py


In [28]:
#Step 1: Find a binary classification dataset. Good places to look: Kaggle, OpenML, UCI Machine Learning Repository.
######   (Part 1) I will be using data_banknote_authentication dataset for my Classification (Part 1)   #####)


#Step 2A: Load the data using pandas.

import pandas as pd
bank_df = pd.read_csv("data_banknote_authentication.txt",header=None)

col_names = ["variance of Wavelet Transformed image (continuous)",
             "skewness of Wavelet Transformed image (continuous)",
             "curtosis of Wavelet Transformed image (continuous)",
             "entropy of image (continuous)",
             "class (integer)"]

bank_df.columns = col_names


#Step 2B: Convert any nominal columns to numerical columns.
#Step 2C: This is also a convenient place to convert target values to 1 or -1.


#print(bank_df.head())
def convenient_convert_target_values(target):
    if target == 0:
        target = -1
    return target

bank_df["class (integer)"] = bank_df["class (integer)"].apply(convenient_convert_target_values)     #Converting 0 to -1 , and 1 to 1


#print(bank_df["class (integer)"])
#print(bank_df.head())


#Step 3: Convert the pandas dataframe to a numpy X and y. Make sure that X has a bias column and that y contains only values of \pm 1±1.

import numpy as np

X = np.ones((bank_df.shape[0],1))     #inserting 1's of bias column
X = np.hstack((X,bank_df[:]))         #combining bias column with everything else
X = X[:,: -1]                         #taking out the last column for y 

#print(X)

y = bank_df["class (integer)"]
y

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
1367    1
1368    1
1369    1
1370    1
1371    1
Name: class (integer), Length: 1372, dtype: int64

In [29]:
#print(y)

#Step 4: Perform a train-test split on the data. You can use your own code or sklearn for this.

from sklearn.model_selection import train_test_split



X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

#Step 5: Scale the data. You can use your own code or sklearn for this. 
#Be sure you scale correctly: First fit the training data to get the scaling parameters, and then apply the scaling to both the training and testing data.

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


X_train = np.c_[np.ones(X_train.shape[0]),X_train]         #re-adding the bias column, because it disappeard after scaler
X_test = np.c_[np.ones(X_test.shape[0]),X_test]            #re-adding the bias column, because it disappeard after scaler

#Step 6:  Train a linear classifier such as logistic regression on the training data. This must be your own code.

def gradient_of_the_pointwise_error_function(x, w, yy):
    return (-yy * x)/(1+np.exp(yy*w.T.dot(x)))
          #(-yy * x)/(1+np.exp(yy*w.T.dot(x)))

def sgd(w,X,y,pw_gradient,eta=0.05,num_epochs=50):
    """parameters: w (initial weight vector)
                   X (data matrix)
                   y (target vector)
                   pw_gradient (pointwise gradient function taking params x,w,yy)"""
    history = [] ## Every time you compute a new w, do history.append(w).
    for j in range(num_epochs):
        shuff = np.random.permutation(X.shape[0])
        Xs = X[shuff]
        Ys = y[shuff]
        for i in range(X.shape[0]):
            xs = Xs[i]
            ys = Ys[i]
            w = w - eta*pw_gradient(xs,w,ys)
            history.append(w)
    return w,np.array(history)

w = np.random.randn(X_train.shape[1])
w, path = sgd(w, X_train, y_train, gradient_of_the_pointwise_error_function,eta=0.001,num_epochs=200 )

#Step 7: Apply your model to the test data to get predictions y hat
yhat = np.sign(X_test.dot(w))

#Step 8: Evaluate the accuracy of your model. You can use simply the proportion of correct predictions.
def E_in(X,y,w):
    mc = (np.sign(X.dot(w)) != y)
    return np.sum(mc)/len(mc)

print("E_in = {}".format(E_in(X_test, y_test, w)))

E_in = 0.09620991253644315


In [30]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.967930029154519

## PART 1 (step 9)

In [31]:
#Step 9: (Optional) Try doing some transformations of features to improve accuracy. After you add or modify features (on the whole dataset) start again at step 4.


#Step 4: Perform a train-test split on the data. You can use your own code or sklearn for this.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

#Step 5: Scale the data. You can use your own code or sklearn for this. 
#Be sure you scale correctly: First fit the training data to get the scaling parameters, and then apply the scaling to both the training and testing data.


scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


X_train = np.c_[np.ones(X_train.shape[0]),X_train]         #re-adding the bias column, because it disappeard after scaler
X_test = np.c_[np.ones(X_test.shape[0]),X_test]            #re-adding the bias column, because it disappeard after scaler

#Step 6:  Train a linear classifier such as logistic regression on the training data. This must be your own code.

def gradient_of_the_pointwise_error_function(x, w, yy):
    return (-yy * x)/(1+np.exp(yy*w.T.dot(x)))


def sgd(w,X,y,pw_gradient,eta=0.05,num_epochs=50):
    """parameters: w (initial weight vector)
                   X (data matrix)
                   y (target vector)
                   pw_gradient (pointwise gradient function taking params x,w,yy)"""
    history = [] ## Every time you compute a new w, do history.append(w).
    for j in range(num_epochs):
        shuff = np.random.permutation(X.shape[0])
        Xs = X[shuff]
        Ys = y[shuff]
        for i in range(X.shape[0]):
            xs = Xs[i]
            ys = Ys[i]
            w = w - eta*pw_gradient(xs,w,ys)
            history.append(w)
    return w,np.array(history)

w = np.random.randn(X_train.shape[1])
w, path = sgd(w, X_train, y_train, gradient_of_the_pointwise_error_function)

#Step 7: Apply your model to the test data to get predictions y hat
yhat = np.sign(X_test.dot(w))

#Step 8: Evaluate the accuracy of your model. You can use simply the proportion of correct predictions.
def E_in(X,y,w):
    mc = (np.sign(X.dot(w)) != y)
    return np.sum(mc)/len(mc)

print("E_in = {}".format(E_in(X_test, y_test, w)))

E_in = 0.014577259475218658


## Part 2 (step 1 - 8)

In [32]:
#Step 1: Find a binary classification dataset. Good places to look: Kaggle, OpenML, UCI Machine Learning Repository.
######   (Part 2) I will be using data_banknote_authentication dataset for my linear regression (Part 2)   #####)
#Step 2A: Load the data using pandas.
import pandas as pd
day_df = pd.read_csv("day.csv")

#Step 2B: Convert any nominal columns to numerical columns.
#Step 2C: This is also a convenient place to convert target values to 1 or -1.

#because it's inconvenient to work with and also kind of redundant.
#You should probably pick one of either casual, registered or cnt to be the target, and drop the other two.
day_df = day_df.drop(columns=['dteday','casual','registered'])

#print(day_df.head())

#Step 3: Convert the pandas dataframe to a numpy X and y. Make sure that X has a bias column and that y contains only values of \pm 1±1.
import numpy as np

X = np.ones((day_df.shape[0],1))     #inserting 1's of bias column
X = np.hstack((X,day_df[:]))         #combining bias column with everything else
X = X[:,: -1]
y = day_df['cnt']

#Step 4: Perform a train-test split on the data. You can use your own code or sklearn for this.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

#Step 5: Scale the data. You can use your own code or sklearn for this. 
#Be sure you scale correctly: First fit the training data to get the scaling parameters, and then apply the scaling to both the training and testing data.
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


X_train = np.c_[np.ones(X_train.shape[0]),X_train]         #re-adding the bias column, because it disappeard after scaler
X_test = np.c_[np.ones(X_test.shape[0]),X_test]            #re-adding the bias column, because it disappeard after scaler

#Step 6:  Train a linear classifier such as logistic regression on the training data. This must be your own code.

def pw_linreg_grad(x,w,yy):
    """parameters: x,w,yy  (instance, weight vector, truth value = +-1)
       return:  new w
       This is the gradient for MSE error.
       The mathematical formula can be found above.
       w = (np.linalg.pinv(X_train).dot(y_train))
       w=w-eta*gradient(w,X,y)
    """
    return 2*(w.T.dot(x)-yy)*x

def sgd(w,X,y,pw_gradient,eta=0.05,num_epochs=50):
    """parameters: w (initial weight vector)
                   X (data matrix)
                   y (target vector)
                   pw_gradient (pointwise gradient function taking params x,w,yy)"""
    history = [] ## Every time you compute a new w, do history.append(w).
    for j in range(num_epochs):
        shuff = np.random.permutation(X.shape[0])
        Xs = X[shuff]
        Ys = y[shuff]
        for i in range(X.shape[0]):
            xs = Xs[i]
            ys = Ys[i]
            w = w - eta*pw_gradient(xs,w,ys)
            history.append(w)
    return w,np.array(history)

def R2(y,yhat):
    """parameters: y, yhat"""
    ## You complete me
    ybar = np.mean(y)
    """  SStot = np.sum((y-yhat)**2)
    SSres = np.sum((y-ybar)**2)"""
    SSres = np.sum((y-yhat)**2)
    SStot = np.sum((y-ybar)**2)
    R2= 1-(SSres/SStot)
    return R2
    #ybar = np.mean(y)
    #r2 = sum((y-yhat)**2)/sum((y-ybar)**2)
    #r2 = 1-r2
    #return r2
w = np.random.randn(X_train.shape[1])
w, path = sgd(w, X_train, y_train, pw_linreg_grad)

#Step 7: Apply your model to the test data to get predictions y hat
yhat_test = X_test.dot(w)

#Step 8: Evaluate the accuracy of your model. You can use simply the proportion of correct predictions.
print("Solution from SGD")
print("w = {},\nR2_test(w) (sgd) = {}".format(w,R2(yhat_test ,y_test)))

Solution from SGD
w = [ 1.74614438e+03 -4.52906507e-01 -1.18069179e+03  1.25364782e+03
  2.12766321e+03  1.88835464e+02 -2.83905076e+02  6.38602991e+02
  3.70121221e+02 -7.42593035e+02  2.25344926e+03  1.51749088e+03
 -1.00832447e+03 -9.59336589e+02],
R2_test(w) (sgd) = 0.5773020639881477


## Part 2  (step 9)

In [33]:
#Step 1: Find a binary classification dataset. Good places to look: Kaggle, OpenML, UCI Machine Learning Repository.
######   (Part 2) I will be using day.csv dataset for my linear (Part 2)   #####)
#Step 2A: Load the data using pandas.
import pandas as pd
day_df = pd.read_csv("day.csv")

#Step 2B: Convert any nominal columns to numerical columns.
#Step 2C: This is also a convenient place to convert target values to 1 or -1.

#because it's inconvenient to work with and also kind of redundant.
#You should probably pick one of either casual, registered or cnt to be the target, and drop the other two.
day_df = day_df.drop(columns=['dteday','casual','registered'])

#print(day_df.head())

#Step 3: Convert the pandas dataframe to a numpy X and y. Make sure that X has a bias column and that y contains only values of \pm 1±1.
import numpy as np

X = np.ones((day_df.shape[0],1))     #inserting 1's of bias column
X = np.hstack((X,day_df[:]))         #combining bias column with everything else
X = X[:,: -1]
y = day_df['cnt']


#Step 4: Perform a train-test split on the data. You can use your own code or sklearn for this.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

#Step 5: Scale the data. You can use your own code or sklearn for this. 
#Be sure you scale correctly: First fit the training data to get the scaling parameters, and then apply the scaling to both the training and testing data.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train = np.c_[np.ones(X_train.shape[0]),X_train]         #re-adding the bias column, because it disappeard after scaler
X_test = np.c_[np.ones(X_test.shape[0]),X_test]            #re-adding the bias column, because it disappeard after scaler

#Step 6:  Train a linear classifier such as logistic regression on the training data. This must be your own code.
def pw_linreg_grad(x,w,yy):
    """parameters: x,w,yy  (instance, weight vector, truth value = +-1)
       return:  new w
       This is the gradient for MSE error.
       The mathematical formula can be found above.
       w = (np.linalg.pinv(X_train).dot(y_train))
       w=w-eta*gradient(w,X,y)
    """
    return 2*(w.T.dot(x)-yy)*x

def sgd(w,X,y,pw_gradient,eta=0.05,num_epochs=50):
    """parameters: w (initial weight vector)
                   X (data matrix)
                   y (target vector)
                   pw_gradient (pointwise gradient function taking params x,w,yy)"""
    history = [] ## Every time you compute a new w, do history.append(w).
    for j in range(num_epochs):
        shuff = np.random.permutation(X.shape[0])
        Xs = X[shuff]
        Ys = y[shuff]
        for i in range(X.shape[0]):
            xs = Xs[i]
            ys = Ys[i]
            w = w - eta*pw_gradient(xs,w,ys)
            history.append(w)
    return w,np.array(history)

def R2(y,yhat):
    """parameters: y, yhat"""
    ## You complete me
    ybar = np.mean(y)
    """  SStot = np.sum((y-yhat)**2)
    SSres = np.sum((y-ybar)**2)"""
    SSres = np.sum((y-yhat)**2)
    SStot = np.sum((y-ybar)**2)
    R2= 1-(SSres/SStot)
    return R2
    #ybar = np.mean(y)
    #r2 = sum((y-yhat)**2)/sum((y-ybar)**2)
    #r2 = 1-r2
    #return r2

w = np.random.randn(X_train.shape[1])
w, path = sgd(w, X_train, y_train, pw_linreg_grad)

#Step 7: Apply your model to the test data to get predictions y hat
yhat_test = X_test.dot(w)

#Step 8: Evaluate the accuracy of your model. You can use simply the proportion of correct predictions.
print("Solution from SGD")
print("w = {},\nR2_test(w) (sgd) = {}".format(w,R2(yhat_test ,y_test)))

Solution from SGD
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan],
R2_test(w) (sgd) = nan


/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:51: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:51: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:66: RuntimeWarning: invalid value encountered in subtract
